# ⏱️ Snowflake Trail - Step 4: Anomaly Detection Setup

---
scheduled Alerts to check for anomalies and log them to the event table

* Tasks run duration
* Task run frequency
* Pipe copy frequency
* Pipe rows ingested
* Dynamic Table rows updated

### Example: numeric value timeseries anomaly

- we can query the hourly credit usage of our serverless alert and see if there are statistical outliers in that history. 
- we use the rounded timestamp column for sorting
- the credits value as the numeric value which we want to analyze
- for each row calculate both average and standard deviation of the values in the previous 50 rows
- calculate the Z-score as ((current value - average value) / standard deviation) for each row
- return all rows with a Z-score above 3

In [ ]:
-- identifying outliers in the hourly credit consumption history of our serverless NEW_ERRORS alert

with 
RECORDS as (
    select
        date_trunc(hour, START_TIME) as REC_TIMESTAMP,
        sum(CREDITS_USED) as REC_VALUE,
        row_number() over (order by REC_TIMESTAMP desc) as REC_NUM
    from 
        table(SNOWTRAIL_DEMO.INFORMATION_SCHEMA.SERVERLESS_ALERT_HISTORY(
            DATE_RANGE_START => current_date - 14
            ))
    where
        ALERT_NAME = 'NEW_ERRORS'
    group by
        REC_TIMESTAMP
    ),
STATS as(
    select
        REC_NUM,
        REC_TIMESTAMP,
        REC_VALUE,
        avg(REC_VALUE) over (order by REC_TIMESTAMP rows between 50 preceding and 1 preceding) as PREV_50_AVG,
        stddev(REC_VALUE) over (order by REC_TIMESTAMP rows between 50 preceding and 1 preceding) as PREV_50_STDDEV,
        abs(REC_VALUE - PREV_50_AVG) / nullif(PREV_50_STDDEV, 0) as Z_SCORE
    from
        RECORDS 
    )
select 
    REC_TIMESTAMP as HOUR_BUCKET,
    REC_VALUE as CREDITS,
    PREV_50_AVG,
    Z_SCORE
from 
    STATS
where
    abs(Z_SCORE) > 3
order by
    REC_TIMESTAMP desc
;

# 1. Task anomalies
 
## 1.1 Task run duration anomaly

For each Task in the selected Database that ran successfully we get the durations from the avaiable previous (max) 50 runs. 
Then calculate the average and standard deviation for each Task.
Then we compare the duration of each new Task run to the standard deviation of its previous runs and return the name if it is an outlier (Z-score over 3).

In [ ]:
create or replace alert SNOWTRAIL_DEMO.OBSERV.TASKS_RUN_DURATION_ANOMALY
--- no warehouse selected to run serverless
schedule = '360 minutes' 
comment = 'Duration outliers in this database'
if (exists(
        with
        RUN_HISTORY as(
            select
                NAME,
                SCHEMA_NAME,
                DATABASE_NAME,
                SCHEDULED_TIME,
                timediff(seconds, QUERY_START_TIME, COMPLETED_TIME) as DURATION,
                avg(DURATION) over (partition by SCHEMA_NAME, NAME order by SCHEDULED_TIME rows between 50 preceding and 1 preceding) as PREV_50_AVG,
                stddev(DURATION) over (partition by SCHEMA_NAME, NAME order by SCHEDULED_TIME rows between 50 preceding and 1 preceding) as PREV_50_STDDEV,
                abs(DURATION - PREV_50_AVG) / nullif(PREV_50_STDDEV, 0) as Z_SCORE
            from
                table(SNOWTRAIL_DEMO.INFORMATION_SCHEMA.TASK_HISTORY(
                        SCHEDULED_TIME_RANGE_START => (
                            coalesce(
                                SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME(),
                                timeadd('DAY', -7, current_timestamp))                              -- if last check is beyond history retention period then use last week instead
                            ),     
                        SCHEDULED_TIME_RANGE_END => SNOWFLAKE.ALERT.SCHEDULED_TIME(),               -- considering only past runs
                        RESULT_LIMIT => 10000))
            where
                SCHEMA_NAME is not null     --- ignoring nested Tasks
                and STATE = 'SUCCEEDED'
        )        
        select
            NAME as TASK_NAME,
            concat(DATABASE_NAME,'.',SCHEMA_NAME) as DB_SCHEMA,
            SCHEDULED_TIME,
            DURATION AS DURATION_IN_S,
            PREV_50_AVG
        from
            RUN_HISTORY
        where
            Z_SCORE > 3                 -- threshold for outliers
        order by
            SCHEDULED_TIME desc
          )
    )
    
then
    begin
        let TASK_DURATION_ANOMALIES resultset := (
            select * from table(result_scan(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID())));      -- get query ID from condition
        
        for RECORD in TASK_DURATION_ANOMALIES do    
            let MESSAGE string := ('Task '||RECORD.TASK_NAME||' in '||RECORD.DB_SCHEMA||' ran for '||RECORD.DURATION_IN_S||' compared to an avg runtime of '||RECORD.PREV_50_AVG||'.');
              
            let WARN_MESSAGE string := ('{"state":"ANOMALY_DETECTED", "message":"'||:MESSAGE||'"} ');  -- add state to json string
            
            select SNOWTRAIL_DEMO.OBSERV.WARN_LOG(:WARN_MESSAGE);       -- using custom logger function from notebook 3
        end for;
    end;
;

In [ ]:
alter alert SNOWTRAIL_DEMO.OBSERV.TASKS_RUN_DURATION_ANOMALY resume;

execute alert SNOWTRAIL_DEMO.OBSERV.TASKS_RUN_DURATION_ANOMALY;

## 1.2. Task run frequency anomalies

Similar to the first example for each Task that ran successfully we now take the time since the previous run and compare it to the previous (max) 50 runs. 
Then calculate the average and standard deviation for each Task.
Then we compare the time diff of each new Task run to the standard deviation of its previous runs and return the name if it is an outlier (Z-score over 3).

In [ ]:
create or replace alert SNOWTRAIL_DEMO.OBSERV.TASKS_RUN_FREQUENCY_ANOMALY
--- no warehouse selected to run serverless
schedule = '360 minutes'
comment = 'Frequency outliers in this database'
if (exists(
        with
        DELTA as(
            select
                NAME,
                SCHEMA_NAME,
                DATABASE_NAME,
                QUERY_START_TIME,
                lead(QUERY_START_TIME) over (partition by SCHEMA_NAME, NAME order by QUERY_START_TIME) as PREV_START_TIME
            from
                table(SNOWTRAIL_DEMO.INFORMATION_SCHEMA.TASK_HISTORY(
                        SCHEDULED_TIME_RANGE_START => (
                            coalesce(
                                SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME(),
                                timeadd('DAY', -7, current_timestamp))                              -- if last check is beyond history retention period then use last week instead
                            ),  
                        SCHEDULED_TIME_RANGE_END => SNOWFLAKE.ALERT.SCHEDULED_TIME(),                   -- considering only past runs
                        RESULT_LIMIT => 10000))
            where
                SCHEMA_NAME is not null     --- ignoring nested Tasks
                and STATE = 'SUCCEEDED'
        ),
        RUN_HISTORY as(
            select
                NAME,
                SCHEMA_NAME,
                QUERY_START_TIME,
                timediff(seconds, QUERY_START_TIME, PREV_START_TIME) as START_TIME_DELTA,
                avg(START_TIME_DELTA) over (partition by SCHEMA_NAME, NAME order by QUERY_START_TIME rows between 50 preceding and 1 preceding) as PREV_50_AVG,
                stddev(START_TIME_DELTA) over (partition by SCHEMA_NAME, NAME order by QUERY_START_TIME rows between 50 preceding and 1 preceding) as PREV_50_STDDEV,
                abs(START_TIME_DELTA - PREV_50_AVG) / nullif(PREV_50_STDDEV, 0) as Z_SCORE
            from
                DELTA
        )        
        select
            NAME as TASK_NAME,
            SCHEMA_NAME,
            -- QUERY_START_TIME,
            START_TIME_DELTA AS START_TIME_DELTA_IN_S,
            PREV_50_AVG,
            -- Z_SCORE
        from
            RUN_HISTORY
        where
            Z_SCORE > 3       -- threshold for outliers
        order by
            QUERY_START_TIME desc
          )
    )
    
then   
    begin
        let TASK_FREQUENCY_ANOMALIES resultset := (
            select * from table(result_scan(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID())));      -- get query ID from condition
        
        for RECORD in TASK_FREQUENCY_ANOMALIES do    
            let MESSAGE string := ('Task '||RECORD.TASK_NAME||' in '||RECORD.SCHEMA_NAME||' did NOT run for '||RECORD.START_TIME_DELTA_IN_S||' compared to an avg frequency of '||RECORD.PREV_50_AVG||'.');
              
            let WARN_MESSAGE string := ('{"state":"ANOMALY_DETECTED", "message":"'||:MESSAGE||'"} ');  -- add state to json string
            
            select SNOWTRAIL_DEMO.OBSERV.WARN_LOG(:WARN_MESSAGE);       -- using custom logger function from notebook 3
        end for;
    end;
;

In [ ]:
alter alert SNOWTRAIL_DEMO.OBSERV.TASKS_RUN_FREQUENCY_ANOMALY resume;

execute alert SNOWTRAIL_DEMO.OBSERV.TASKS_RUN_FREQUENCY_ANOMALY;

# 2. Pipe Anomalies

## 2.1. Pipe copy frequency anomalies

In a similar way we can take the timestamp of each successfully copy from a defined Pipe. 
Then we take the time since the previous copy and compare it to gaps between previous copies. 
Then calculate the average and standard deviation and compare the time diff of each new Copy to the standard deviation of the previous gaps and return the timestamp if it is an outlier (Z-score over 3).

In [ ]:
create or replace alert SNOWTRAIL_DEMO.OBSERV.PIPE_COPY_FREQUENCY_ANOMALY
--- no warehouse selected to run serverless
schedule = '360 minutes'
comment = 'Frequency outliers for Pipe LOAD_STEADY_WEATHER'
if (exists(
        with
        DELTA as(
            select
                PIPE_NAME,
                PIPE_SCHEMA_NAME,
                PIPE_CATALOG_NAME,
                LAST_LOAD_TIME,
                lead(LAST_LOAD_TIME) over (order by LAST_LOAD_TIME) as PREV_LOAD_TIME
            from
                table(SNOWTRAIL_DEMO.INFORMATION_SCHEMA.COPY_HISTORY(
                        TABLE_NAME => 'SNOWTRAIL_DEMO.PIPELINE.IMPORTED_WEATHER',       -- select name of Pipe target table
                        START_TIME => (
                            coalesce(
                                SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME(),
                                timeadd('DAY', -7, current_timestamp))                  -- if last check is beyond history retention period then use last week instead
                            )  
                        ))
            where
                PIPE_NAME = 'LOAD_DAILY_WEATHER'                                        -- select name of a Pipe
        ),
        COPY_HISTORY as(
            select
                PIPE_NAME,
                PIPE_SCHEMA_NAME,
                PIPE_CATALOG_NAME,
                LAST_LOAD_TIME,
                timediff(seconds, LAST_LOAD_TIME, PREV_LOAD_TIME) as TIME_SINCE_LAST_LOAD,
                avg(TIME_SINCE_LAST_LOAD) over (order by LAST_LOAD_TIME rows between 50 preceding and 1 preceding) as PREV_50_AVG,
                stddev(TIME_SINCE_LAST_LOAD) over (order by LAST_LOAD_TIME rows between 50 preceding and 1 preceding) as PREV_50_STDDEV,
                abs(TIME_SINCE_LAST_LOAD - PREV_50_AVG) / nullif(PREV_50_STDDEV, 0) as Z_SCORE
            from
                DELTA
        )        
        select
            PIPE_NAME,
            concat(PIPE_CATALOG_NAME,'.',PIPE_SCHEMA_NAME) as DB_SCHEMA,
            -- LAST_LOAD_TIME,
            TIME_SINCE_LAST_LOAD AS TIME_SINCE_LAST_LOAD_IN_S,
            PREV_50_AVG,
            -- Z_SCORE
        from
            COPY_HISTORY
        where
            Z_SCORE > 3       -- threshold for outliers
        order by
            LAST_LOAD_TIME desc
          )
    )
    
then
    begin
        let COPY_FREQUENCY_ANOMALIES resultset := (
            select * from table(result_scan(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID())));      -- get query ID from condition
        
        for RECORD in COPY_FREQUENCY_ANOMALIES do    
            let MESSAGE string := ('Pipe '||RECORD.PIPE_NAME||' in '||RECORD.DB_SCHEMA||' did NOT load now data for '||RECORD.TIME_SINCE_LAST_LOAD_IN_S||' compared to an avg frequency of '||RECORD.PREV_50_AVG||'.');
              
            let WARN_MESSAGE string := ('{"state":"ANOMALY_DETECTED", "message":"'||:MESSAGE||'"} ');  -- add state to json string
            
            select SNOWTRAIL_DEMO.OBSERV.WARN_LOG(:WARN_MESSAGE);       -- using custom logger function from notebook 3
        end for;
    end;
;

In [ ]:
alter alert SNOWTRAIL_DEMO.OBSERV.PIPE_COPY_FREQUENCY_ANOMALY resume;

execute alert SNOWTRAIL_DEMO.OBSERV.PIPE_COPY_FREQUENCY_ANOMALY;

## 2.2. Pipe ingestion row count anomaly

Similar to the ingestion frequency we can now check for row count anomalies from our Pipe: 

In [ ]:
create or replace alert SNOWTRAIL_DEMO.OBSERV.PIPE_ROW_COUNT_ANOMALY
--- no warehouse selected to run serverless
schedule = '360 minutes'
comment = 'Row count outliers for Pipe LOAD_STEADY_WEATHER'
if (exists(
        with 
        COPY_HISTORY as (
            select
                PIPE_NAME,
                CATALOG_NAME,
                SCHEMA_NAME,
                LAST_LOAD_TIME,
                ROW_COUNT as ROWS_COPIED,
                avg(ROWS_COPIED) over (order by LAST_LOAD_TIME rows between 50 preceding and 1 preceding) as PREV_50_AVG,
                stddev(ROWS_COPIED) over (order by LAST_LOAD_TIME rows between 50 preceding and 1 preceding) as PREV_50_STDDEV,
                abs(ROWS_COPIED - PREV_50_AVG) / nullif(PREV_50_STDDEV, 0) as Z_SCORE
            from
                table(SNOWTRAIL_DEMO.INFORMATION_SCHEMA.COPY_HISTORY(
                        TABLE_NAME => 'SNOWTRAIL_DEMO.PIPELINE.IMPORTED_WEATHER',       -- select name of Pipe target table
                        START_TIME => (
                            coalesce(
                                SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME(),
                                timeadd('DAY', -7, current_timestamp))                  -- if last check is beyond history retention period then use last week instead
                            )  
                        ))
            where
                PIPE_NAME = 'LOAD_DAILY_WEATHER'                                        -- select name of a Pipe
        )
        select
            PIPE_NAME,
            concat(CATALOG_NAME,'.',SCHEMA_NAME) as DB_SCHEMA,
            LAST_LOAD_TIME,
            ROWS_COPIED,
            PREV_50_AVG,
            -- Z_SCORE
        from
            COPY_HISTORY
        where
            Z_SCORE > 3       -- threshold for outliers
          )
    )
    
then
    begin
        let COPY_ROWS_ANOMALIES resultset := (
            select * from table(result_scan(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID())));      -- get query ID from condition
        
        for RECORD in COPY_ROWS_ANOMALIES do    
            let MESSAGE string := ('Pipe '||RECORD.PIPE_NAME||' in '||RECORD.DB_SCHEMA||' loaded a file with '||RECORD.ROWS_COPIED||' rows compared to an avg row-count of '||RECORD.PREV_50_AVG||'.');
              
            let WARN_MESSAGE string := ('{"state":"ANOMALY_DETECTED", "message":"'||:MESSAGE||'"} ');  -- add state to json string
            
            select SNOWTRAIL_DEMO.OBSERV.WARN_LOG(:WARN_MESSAGE);       -- using custom logger function from notebook 3
        end for;
    end;
;

⚠️ Note that INFORMATION_SCHEMA.COPY_HISTORY() requires a target table as argument. So we can not just get all Pipe copies from one query like we can for Task runs and Dynamic Table refreshes.
If we want to set up one Alert convering row-count anomalies for all Pipes in our database we would have to look up the target tables for each Pipe and add a loop to our query.

In [ ]:
alter alert SNOWTRAIL_DEMO.OBSERV.PIPE_ROW_COUNT_ANOMALY resume;

execute alert SNOWTRAIL_DEMO.OBSERV.PIPE_ROW_COUNT_ANOMALY;

# 3. Dynamic Tables Anomalies

## 3.1. Dynamic Table refresh row-change anomaly

Similar to the row count check for Pipe ingestions we can do the same for Dynamic Table refreshes:

In [ ]:
create or replace alert SNOWTRAIL_DEMO.OBSERV.DT_REFRESH_ROW_COUNT_ANOMALY
--- no warehouse selected to run serverless
schedule = '360 minutes'
if (exists(
        with 
        REFRESH_HISTORY as (
            select 
                NAME as DT_NAME,
                DATABASE_NAME,
                SCHEMA_NAME,
                concat(DATABASE_NAME,'.',SCHEMA_NAME,'.',NAME) as FULL_NAME,
                REFRESH_START_TIME, 
                STATISTICS:numCopiedRows as UPDATED_ROWS,
                avg(UPDATED_ROWS) over (partition by FULL_NAME order by REFRESH_START_TIME rows between 50 preceding and 1 preceding) as PREV_50_AVG,
                stddev(UPDATED_ROWS) over (partition by FULL_NAME order by REFRESH_START_TIME rows between 50 preceding and 1 preceding) as PREV_50_STDDEV,
                abs(UPDATED_ROWS - PREV_50_AVG) / nullif(PREV_50_STDDEV, 0) as Z_SCORE
            from 
                table(SNOWTRAIL_DEMO.INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY(
                        DATA_TIMESTAMP_START => (
                            coalesce(
                                SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME(),
                                timeadd('DAY', -7, current_timestamp))                  -- if last check is beyond history retention period then use last week instead
                            ), 
                        DATA_TIMESTAMP_END => SNOWFLAKE.ALERT.SCHEDULED_TIME(),
                        RESULT_LIMIT => 10000
                    )) 
            where 
                UPDATED_ROWS > 0
                and REFRESH_TRIGGER = 'SCHEDULED'
            order by
                DATA_TIMESTAMP desc
        )
        select
            DT_NAME,
            concat(DATABASE_NAME,'.',SCHEMA_NAME) as DB_SCHEMA,
            REFRESH_START_TIME,
            UPDATED_ROWS,
            PREV_50_AVG,
            Z_SCORE
        from
            REFRESH_HISTORY
        where
            Z_SCORE > 3       -- threshold for outliers
        )
    )
then
    begin
        let REFRESH_ROWS_ANOMALIES resultset := (
            select * from table(result_scan(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID())));      -- get query ID from condition
        
        for RECORD in REFRESH_ROWS_ANOMALIES do    
            let MESSAGE string := ('Dynamic Table '||RECORD.DT_NAME||' in '||RECORD.DB_SCHEMA||' refreshed with '||RECORD.UPDATED_ROWS||' rows changed compared to an avg of '||RECORD.PREV_50_AVG||' rows.');
              
            let WARN_MESSAGE string := ('{"state":"ANOMALY_DETECTED", "message":"'||:MESSAGE||'"} ');  -- add state to json string
            
            select SNOWTRAIL_DEMO.OBSERV.WARN_LOG(:WARN_MESSAGE);       -- using custom logger function from notebook 3
        end for;
    end;
;

In [ ]:
alter alert SNOWTRAIL_DEMO.OBSERV.DT_REFRESH_ROW_COUNT_ANOMALY resume;

execute alert SNOWTRAIL_DEMO.OBSERV.DT_REFRESH_ROW_COUNT_ANOMALY;

## Balancing Cost and Latency

The examples above should serve you as templates for your own pipelines and projects. Keep in mind that they are all scoped to a specific Database. You can also expand them to the entire account by querying SNOWFLAKE.INFORMATION_SCHEMA or reduce the scope to a specific Schema.
Also think about what a good schedule would be for your projects. You can run checks ever minute, every hour, every day - depending on the throughput of your pipelines and your business needs. 

To keep an eye on you cost and then find your balance you can check **INFORMATION_SCHEMA.SERVERLESS_ALERT_HISTORY**:

...or set up an anomaly detection Alert on your Alert spent 🙃

In [ ]:
import streamlit as st
import pandas as pd
import altair as alt
session = get_active_session()

st.header('Serverless Alerts Costs')

SERVERLESS_CREDITS = session.sql("""
                select
                    ALERT_NAME,
                    to_date(START_TIME) as DS,
                    sum(CREDITS_USED) as CREDITS_SPENT
                from 
                    table(SNOWTRAIL_DEMO.INFORMATION_SCHEMA.SERVERLESS_ALERT_HISTORY(
                        DATE_RANGE_START => current_date - 7
                    ))
                group by 
                    ALERT_NAME,
                    DS
                """).to_pandas()

CHART = alt.Chart(SERVERLESS_CREDITS).mark_bar(size=30).encode(
        x=alt.X('DS:T', axis=alt.Axis(title= None)), 
        y=alt.Y('CREDITS_SPENT:Q', axis=alt.Axis(title='CREDITS')), 
        color=alt.Color('ALERT_NAME:N')
        ).properties(height=360, width=720)

st.altair_chart(CHART)